In [20]:
import os.path as osp
import matplotlib.pyplot as plt
import time
import torch
import numpy as np
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import pandas as pd
import random
import json
import torchvision
import torch.nn.init as init
import cv2

%matplotlib inline

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [4]:
def make_datapath_list(rootpath):
    """学習検証の画像データとアノテーションデータ，マスクデータへのファイルパスリストを作成"""
    
    #load JSON-file for annotation
    json_path=osp.join(rootpath,'COCO.json')
    with open(json_path) as data_file:
        data_this=json.load(data_file)
        data_json=data_this['root']
        
    #store index
    num_sampels=len(data_json)
    train_indexes=[]
    val_indexes=[]
    for count in range(num_sampels):
        if data_json[count]['isValidation']!=0:
            val_indexes.append(count)
        else:
            train_indexes.append(count)
            
    #store file paths to images
    train_img_list=list()
    val_img_list=list()
    
    for idx in train_indexes:
        img_path=osp.join(rootpath,data_json[idx]['img_paths']) #'train2014/COCO_train2014_000000000036.jpg'
        train_img_list.append(img_path)
    for idx in val_indexes:
        img_path=osp.join(rootpath,data_json[idx]['img_paths'])
        val_img_list.append(img_path)
        
    #store paths to mask data
    train_mask_list=[]
    val_mask_list=[]
    
    for idx in train_indexes:
        img_idx=data_json[idx]['img_paths'][-16:-4] #000000000036
        anno_path="./data/mask/train2014/mask_COCO_train2014_"+img_idx+'.jpg'
        train_mask_list.append(anno_path)
    for idx in val_indexes:
        img_idx=data_json[idx]['img_paths'][-16:-4]
        anno_path="./data/mask/val2014/mask_COCO_val2014_"+img_idx+'.jpg'
        val_mask_list.append(anno_path)
        
    #store annotation data
    train_meta_list=list()
    val_meta_list=list()
    
    for idx in train_indexes:
        train_meta_list.append(data_json[idx])
    for idx in val_indexes:
        val_meta_list.append(data_json[idx])
        
    return train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list

#データ処理のクラスとデータオーギュメンテーションのクラスをimportする
from utils.data_augumentation import Compose,get_anno,add_neck,aug_scale,aug_rotate,aug_croppad,aug_croppad,aug_flip,remove_illegal_joint,\
Normalize_Tensor,no_Normalize_Tensor

class DataTransform():
    """
    画像とマスク，アノテーションの前処理クラス
    学習時と推論時で異なる動作をする
    学習時はdata augmentationをする
    """
    
    def __init__(self):
        
        self.data_transform={
            'train':Compose([
                get_anno(), #store annotation from JSON to dict
                add_neck(), #align anno data, add neck anno data
                aug_scale(), #scaling
                aug_rotate(), #rotation
                aug_croppad(), #trimming
                aug_flip(), #flip horizontal
                remove_illegal_joint(), #remove the anno data out of img
                #Normlize_Tensor()
                no_Normalize_Tensor() #only in this chapter, no-Normalization
            ]),
            'val':Compose([
                #omit in this textbook
            ])
        }
        
    def __call__(self,phase,meta_data,img,mask_miss):
        """
        Parameters
        phase:'train'or'val' determine the mode of preprocess
        """
        meta_data,img,mask_miss=self.data_transform[phase](meta_data,img,mask_miss)
        
        return meta_data,img,mask_miss
    
from utils.dataloader import get_ground_truth

class COCOkeypointsDataset(data.Dataset):
    """
    MSCOCOのCocokeypointsのdatasetを作成するクラス．pytorchのDatasetクラスを継承
    
    Attributes
    img_list : list 画像のパスを格納したリスト
    anno_list : list アノテーションへのパスを格納したリスト
    pahse : 'train' or 'test'
    transform : 前処理クラスのインスタンス
    """
    
    def __init__(self,img_list,mask_list,meta_list,phase,transform):
        super(COCOkeypointsDataset,self).__init__()
        self.img_list=img_list
        self.mask_list=mask_list
        self.meta_list=meta_list
        self.phase=phase
        self.transform=transform
        
    def __len__(self):
        #画像の枚数を返す
        return len(self.img_list)

    def __getitem__(self,index):
        img,heatmaps,heat_mask,pafs,paf_mask=self.pull_item(index)
        return img,heatmaps,heat_mask,pafs,paf_mask

    def pull_item(self,index):
        #画像のtensor形式のデータ，アノテーション，マスクを取得する

        #1. 画像読み込み
        image_file_path=self.img_list[index]
        img=cv2.imread(image_file_path) #[高さ][幅][色BGR]

        #2. マスクとアノテーション読み込み
        mask_miss=cv2.imread(self.mask_list[index])
        meta_data=self.meta_list[index]

        #3. 画像前処理
        meta_data, img, mask_miss=self.transform(self.phase,meta_data,img,mask_miss)

        #4. 正解アノテーションデータの取得
        mask_miss_numpy=mask_miss.numpy().transpose((1,2,0))
        heat_mask,heatmaps,paf_mask,pafs=get_ground_truth(meta_data,mask_miss_numpy)

        #5. マスクデータはRGBが(1,1,1)か(0,0,0)なので，次元を落とす
        #マスクデータはマスクされている場所は値が0,それ以外は1
        heat_mask=heat_mask[:,:,:,0]
        paf_mask=paf_mask[:,:,:,0]

        #6. チャネルが最後尾にあるので順番を変える
        # exp:paf_mask:torch.Size([46,46,38]) → torch.Size([38,46,46])
        paf_mask=paf_mask.permute(2,0,1)
        heat_mask=heat_mask.permute(2,0,1)
        pafs=pafs.permute(2,0,1)
        heatmaps=heatmaps.permute(2,0,1)

        return img,heatmaps,heat_mask,pafs,paf_mask

In [5]:
class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet,self).__init__()
        
        #Feature module
        self.model0=OpenPose_Feature()
        
        #Stage module
        #PAFs (part affinity fields) side
        self.model1_1=make_OpenPose_block('block1_1')
        self.model2_1=make_OpenPose_block('block2_1')
        self.model3_1=make_OpenPose_block('block3_1')
        self.model4_1=make_OpenPose_block('block4_1')
        self.model5_1=make_OpenPose_block('block5_1')
        self.model6_1=make_OpenPose_block('block6_1')
        
        #confidence heatmap side
        self.model1_2=make_OpenPose_block('block1_2')
        self.model2_2=make_OpenPose_block('block2_2')
        self.model3_2=make_OpenPose_block('block3_2')
        self.model4_2=make_OpenPose_block('block4_2')
        self.model5_2=make_OpenPose_block('block5_2')
        self.model6_2=make_OpenPose_block('block6_2')
        
    def forward(self,x):
        #順伝搬定義
        
        #Feature module
        out1=self.model0(x)
        
        #stage1
        out1_1=self.model1_1(out1) #PAFs side
        out1_2=self.model1_2(out1) #confidence heatmap side
        
        #stage2
        out2=torch.cat([out1_1,out1_2,out1],1) #次元1のチャネルで結合
        out2_1=self.model2_1(out2)
        out2_2=self.model2_2(out2)
        
        #stage3
        out3=torch.cat([out2_1,out2_2,out1],1)
        out3_1=self.model3_1(out3)
        out3_2=self.model3_2(out3)
        
        #stage4
        out4=torch.cat([out3_1,out3_2,out1],1)
        out4_1=self.model4_1(out4)
        out4_2=self.model4_2(out4)
        
        #stage5
        out5=torch.cat([out4_1,out4_2,out1],1)
        out5_1=self.model5_1(out5)
        out5_2=self.model5_2(out5)
        
        #stage6
        out6=torch.cat([out5_1,out5_2,out1],1)
        out6_1=self.model6_1(out6)
        out6_2=self.model6_2(out6)
        
        #損失の計算用に各stageの結果を格納
        saved_for_loss=[]
        saved_for_loss.append(out1_1)
        saved_for_loss.append(out1_2)
        saved_for_loss.append(out2_1)
        saved_for_loss.append(out2_2)
        saved_for_loss.append(out3_1)
        saved_for_loss.append(out3_2)
        saved_for_loss.append(out4_1)
        saved_for_loss.append(out4_2)
        saved_for_loss.append(out5_1)
        saved_for_loss.append(out5_2)
        saved_for_loss.append(out6_1)
        saved_for_loss.append(out6_2)
        
        #最終的なPAFsのout6_1とconfidence heatmapのout6_2，そして，損失計算用に各ステージでのPAFsとheatmapを格納したsaved_for_lossを出力
        #out6_1 : torch.Size([minibatch, 38, 46, 46])
        #out6_2 : torch.Size([minibatch, 19, 46, 46])
        #saved_for_loss : [out1_1, out1_2, ... , out6_2]
        
        return (out6_1, out6_2), saved_for_loss
    
class OpenPose_Feature(nn.Module):
    def __init__(self):
        super(OpenPose_Feature,self).__init__()
        
        #VGG-19の最初10個の畳み込みを使用
        #初めて実行する際はモデルの重みパラメタをダウンロードするため実行に時間がかかる
        vgg19=torchvision.models.vgg19(pretrained=True)
        model={}
        model['block0']=vgg19.features[0:23] #vgg19の最初の10畳み込み層まで
        
        #残りは新たな畳み込み層を2つ用意
        model['block0'].add_module("23", torch.nn.Conv2d(
            512,256,kernel_size=3,stride=1,padding=1))
        model['block0'].add_module("24", torch.nn.ReLU(inplace=True))
        model['block0'].add_module("25", torch.nn.Conv2d(
            256,128,kernel_size=3,stride=1,padding=1))
        model['block0'].add_module("26", torch.nn.ReLU(inplace=True))
        
        self.model=model['block0']
        
    def forward(self,x):
        outputs=self.model(x)
        return outputs
    
def make_OpenPose_block(block_name):
    #コンフィギュレーション変数からOpenPoseのStageモジュールのblockを作成．nn.Moduleではなく，nn.Sequential
    
    #1.コンフィギュレーションの辞書変数blocksを作成し，ネットワークを生成させる
    #最初に全パターンの辞書を用意し，引数block_nameのみを生成する
    blocks={}
    #stage1
    blocks['block1_1']=[{'conv5_1_CPM_L1':[128,128,3,1,1]},
                        {'conv5_2_CPM_L1':[128,128,3,1,1]},
                        {'conv5_3_CPM_L1':[128,128,3,1,1]},
                        {'conv5_4_CPM_L1':[128,512,1,1,0]},
                        {'conv5_5_CPM_L1':[512,38,1,1,0]}]
    
    blocks['block1_2']=[{'conv5_1_CPM_L2':[128,128,3,1,1]},
                        {'conv5_2_CPM_L2':[128,128,3,1,1]},
                        {'conv5_3_CPM_L2':[128,128,3,1,1]},
                        {'conv5_4_CPM_L2':[128,512,1,1,0]},
                        {'conv5_5_CPM_L2':[512,19,1,1,0]}]
    
    #stage2-6
    for i in range(2,7):
        blocks['block%d_1' % i]=[
            {'Mconv1_stage%d_L1' % i:[185,128,7,1,3]},
            {'Mconv2_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv3_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv4_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv5_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv6_stage%d_L1' % i:[128,128,1,1,0]},
            {'Mconv7_stage%d_L1' % i:[128,38,1,1,0]}
        ]
        blocks['block%d_2' % i]=[
            {'Mconv1_stage%d_L2' % i:[185,128,7,1,3]},
            {'Mconv2_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv3_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv4_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv5_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv6_stage%d_L2' % i:[128,128,1,1,0]},
            {'Mconv7_stage%d_L2' % i:[128,19,1,1,0]}
        ]
        
    #引数block_nameのコンフィギュレーション辞書を取り出す
    cfg_dict=blocks[block_name]
    
    #2.コンフィギュレーション内容をリスト変数layersに格納
    layers=[]
    
    #0番目から最後の層までを作成
    for i in range(len(cfg_dict)):
        for k, v in cfg_dict[i].items():
            if 'pool' in k:
                layers+=[nn.MaxPool2d(kernel_size=v[0],stride=v[1],padding=v[2])]
            else:
                conv2d=nn.Conv2d(in_channels=v[0],out_channels=v[1],kernel_size=v[2],stride=v[3],padding=v[4])
                layers+=[conv2d,nn.ReLU(inplace=True)]
                
    #3.layersをSequentialにする
    #ただし，最後にReLUはいらないのでその手前までを使用
    net=nn.Sequential(*layers[:-1])
    
    #4.初期化関数の設定をし，畳み込み層を初期化する
    def _initialize_weights_norm(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                init.normal_(m.weight,std=0.01)
                if m.bias is not None:
                    init.constant_(m.bias,0.0)
                    
    net.apply(_initialize_weights_norm)
    
    return net

In [12]:
#MS COCOのファイルパスリストを作成
train_img_list,train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(rootpath="./data/")

#Dataset作成
#本書ではデータ量の問題から，trainをvalで作成している

train_dataset=COCOkeypointsDataset(val_img_list,val_mask_list,val_meta_list,phase='train',transform=DataTransform())

#今回は簡易な学習とし検証データは作成しない
#val_dataset=COCOkeypointsDataset(val_img_list,val_mask_list,val_meta_list,phase='val',transform=DataTransform())

#DataLoader作成
batch_size=32

train_dataloader=data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

#val_dataloader=data.DataLoader(val_dataset,batch_size=batch_size,shuffle=False)

#辞書型変数にまとめる
#dataloaders_dict={"train":train_dataloader,"val":val_dataloader}
dataloaders_dict={"train":train_dataloader,"val":None}

net=OpenPoseNet()

In [13]:
#損失関数の設定
class OpenPoseLoss(nn.Module):
    #OpenPoseの損失関数のクラスです
    
    def __init__(self):
        super(OpenPoseLoss,self).__init__()
        
    def forward(self,saved_for_loss,heatmap_target,heat_mask,paf_target,paf_mask):
        """
        損失関数の計算
        
        Parameters
        saved_for_loss : OpenPoseNetの出力（リスト）
        heatmap_target : [num_batch,19,46,46]
        heatmap_mask : [num_batch,19,46,46] heatmap画像のmask
        paf_target : [num_batch,38,46,46] 正解のPAFのアノテーション情報
        paf_mask : [num_batch,38,46,46] PAF画像のmask
        
        Returns
        loss : tensor 損失値
        """
        
        total_loss=0
        #ステージごとに計算
        for j in range(6):
            
            #PAFsとheatmapsにおいて，マスクされている部分(paf_mask=0など)は無視する
            #PAFs
            pred1=saved_for_loss[2*j]*paf_mask
            gt1=paf_target.float()*paf_mask
            
            #heatmaps
            pred2=saved_for_loss[2*j+1]*heat_mask
            gt2=heatmap_target.float()*heat_mask
            
            total_loss += F.mse_loss(pred1,gt1,reduction='mean') + F.mse_loss(pred2,gt2,reduction='mean')
            
        return total_loss
    
criterion=OpenPoseLoss()

In [18]:
optimizer=optim.SGD(net.parameters(),lr=1e-2,momentum=0.9,weight_decay=0.0001)

In [16]:
#モデルを学習させる関数を作成

def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
    
    #GPUが使えるか確認
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：",device)
    
    #ネットワークをGPUへ
    net.to(device)
    
    #ネットワークがある程度固定であれば高速化させる
    torch.backends.cudnn.benchmark=True
    
    #画像の枚数
    num_train_imgs=len(dataloaders_dict["train"].dataset)
    batch_size=dataloaders_dict["train"].batch_size
    
    #イテレーションカウンタをセット
    iteration=1
    
    #epoch-loop
    for epoch in range(num_epochs):
        
        #開始時刻
        t_epoch_start=time.time()
        t_iter_start=time.time()
        epoch_train_loss=0.0 #epochの損失和
        epoch_val_loss=0.0 #epochの損失和
        
        print('-------------------')
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-------------------')
        
        #epochごとの訓練と検証のループ
        for phase in ['train','val']:
            if phase=='train':
                net.train()
                optimizer.zero_grad()
                print(' (train) ')
            #今回は検証はスキップ
            else:
                continue
                #net.eval()
                #print('-------------------')
                #print(' (val) ')
                
            #データローダからミニバッチずつ取り出すループ
            for imges,heatmap_target,heat_mask,paf_target,paf_mask in dataloaders_dict[phase]:
                #ミニバッチサイズが1だと，バッチノーマライゼーションでエラーになる
                if imges.size()[0]==1:
                    continue
                    
                #GPUが使えるならGPUにデータを送る
                imges=imges.to(device)
                heatmap_target=heatmap_target.to(device)
                heat_mask=heat_mask.to(device)
                paf_target=paf_target.to(device)
                paf_mask=paf_mask.to(device)
                
                #optimizerを初期化
                optimizer.zero_grad()
                
                #calc forward
                with torch.set_grad_enabled(phase=='train'):
                    #(out6_1,out6_2)は使わないので＿で代替
                    _,saved_for_loss=net(imges)
                    
                    loss=criterion(saved_for_loss,heatmap_target,heat_mask,paf_target,paf_mask)
                    
                    #訓練時はback-prop
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                        
                        if(iteration%10==0):
                            t_iter_finishi=time.time()
                            duration=t_iter_finishi-t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration,loss.item()/batch_size,duration))
                            t_iter_start=time.time()
                        
                        epoch_train_loss+=loss.item()
                        iteration+=1
                    
                    #検証時
                    #else:
                        #epoch_val_loss+=loss.item()
                        
        #epochのphaseごとのlossと正解率
        t_epoch_finishi=time.time()
        print('-------------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch__VAL_Loss:{:.4f}'.format(
            epoch+1,epoch_train_loss/num_train_imgs, 0))
        print('timer: {:.4f} sec'.format(t_epoch_finishi-t_epoch_start))
        t_epoch_start=time.time()
        
    #最後のネットワークを保存
    torch.save(net.state_dict(), './weights/openpose_net_'+str(epoch+1)+'.pth')

In [21]:
#学習・検証
num_epochs=2
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs=num_epochs)

使用デバイス： cuda:0
-------------------
Epoch 1/2
-------------------
 (train) 
イテレーション 10 || Loss: 0.0093 || 10iter: 69.2442 sec.
イテレーション 20 || Loss: 0.0084 || 10iter: 53.8389 sec.
イテレーション 30 || Loss: 0.0070 || 10iter: 44.0771 sec.
イテレーション 40 || Loss: 0.0059 || 10iter: 44.5049 sec.
イテレーション 50 || Loss: 0.0049 || 10iter: 40.5896 sec.
イテレーション 60 || Loss: 0.0041 || 10iter: 36.9888 sec.
イテレーション 70 || Loss: 0.0039 || 10iter: 34.4608 sec.
イテレーション 80 || Loss: 0.0033 || 10iter: 35.2298 sec.
イテレーション 90 || Loss: 0.0028 || 10iter: 34.0990 sec.
イテレーション 100 || Loss: 0.0025 || 10iter: 34.1013 sec.
イテレーション 110 || Loss: 0.0023 || 10iter: 33.2162 sec.
イテレーション 120 || Loss: 0.0021 || 10iter: 35.2386 sec.
イテレーション 130 || Loss: 0.0021 || 10iter: 35.0580 sec.
イテレーション 140 || Loss: 0.0018 || 10iter: 34.1989 sec.
イテレーション 150 || Loss: 0.0018 || 10iter: 33.7657 sec.
-------------------
epoch 1 || Epoch_TRAIN_Loss:0.0043 ||Epoch__VAL_Loss:0.0000
timer: 614.5134 sec
-------------------
Epoch 2/2
-------------------
 (tr